# Learn Sine Function with Neural Network Regression

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

import numpy as np
import matplotlib.pyplot as plt

## Generate Data

Create training data by sampling from a sine function.

In [ ]:
# Generate x and y data
xpts = np.arange(0, 1000, 0.1)
ypts = (np.sin(xpts) + 1) / 2  # Normalize to [0, 1]

# Convert to PyTorch tensors
X = torch.from_numpy(xpts).float().unsqueeze(1)  # Shape: (n_samples, 1)
y = torch.from_numpy(ypts).float().unsqueeze(1)  # Shape: (n_samples, 1)

print(f'X shape: {X.shape}')
print(f'y shape: {y.shape}')

## Define Model

Create a neural network to learn the sine function.

In [ ]:
class SineRegressionNet(nn.Module):
    def __init__(self):
        super(SineRegressionNet, self).__init__()
        self.fc1 = nn.Linear(1, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        return x

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = SineRegressionNet().to(device)

# Print model summary
print(model)
print(f'\nTotal parameters: {sum(p.numel() for p in model.parameters())}')

## Setup Training

Define loss function and optimizer.

In [ ]:
# Loss and optimizer
loss_fn = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Create DataLoader
batch_size = 16
epochs = 200

# Split data into train/validation
train_size = int(0.9 * len(X))
train_indices = torch.randperm(len(X))[:train_size]
val_indices = torch.randperm(len(X))[train_size:]

X_train, y_train = X[train_indices], y[train_indices]
X_val, y_val = X[val_indices], y[val_indices]

train_dataset = TensorDataset(X_train, y_train)
val_dataset = TensorDataset(X_val, y_val)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

## Train Model

Train the model for the specified number of epochs.

In [ ]:
train_losses = []
val_losses = []

for epoch in range(epochs):
    # Training
    model.train()
    train_loss = 0
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        
        # Forward pass
        y_pred = model(X_batch)
        loss = loss_fn(y_pred, y_batch)
        
        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
    
    train_loss /= len(train_loader)
    train_losses.append(train_loss)
    
    # Validation
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            y_pred = model(X_batch)
            loss = loss_fn(y_pred, y_batch)
            val_loss += loss.item()
    
    val_loss /= len(val_loader)
    val_losses.append(val_loss)
    
    if (epoch + 1) % 50 == 0:
        print(f'Epoch {epoch+1}/{epochs}, Train Loss: {train_loss:.6f}, Val Loss: {val_loss:.6f}')

## Plot Training History

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(train_losses, label='Training Loss')
plt.plot(val_losses, label='Validation Loss')
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend()
plt.grid(True)
plt.show()

## Test Predictions

Compare model predictions with ground truth on unseen data.

In [ ]:
# Generate test data
test_x = np.arange(0, 100, 0.1)
test_X = torch.from_numpy(test_x).float().unsqueeze(1).to(device)

# Make predictions
model.eval()
with torch.no_grad():
    test_y_pred = model(test_X).cpu().numpy().flatten()

# Ground truth
test_y_true = (np.sin(test_x) + 1) / 2

# Plot
plt.figure(figsize=(12, 5))
plt.plot(test_x, test_y_pred, label='Predicted', linewidth=2)
plt.plot(test_x, test_y_true, label='True Sine', linewidth=2)
plt.xlabel('x')
plt.ylabel('y')
plt.title('Model Predictions vs Ground Truth')
plt.legend()
plt.grid(True)
plt.show()